In [ ]:
from Chapter_08_MT5_forex_sinal_M10 import *
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import yfinance as yf
import pickle
import ta
from joblib import dump, load
import os
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten



from IPython.display import HTML, Javascript, display

def restart_kernel_and_run_all_cells():               

        display(HTML(
            '''
                <script>
                    code_show = false;
                    function restart_run_all(){
                        IPython.notebook.kernel.restart();
                        setTimeout(function(){
                            IPython.notebook.execute_all_cells();
                        }, 10000)
                    }
                    function code_toggle() {
                        if (code_show) {
                            $('div.input').hide(200);
                        } else {
                            $('div.input').show(200);
                        }
                        code_show = !code_show
                    }
                    code_toggle() 
                    restart_run_all()
                </script>

            '''
        ))



path = "C:/Users/Administrator/Documents/UDEMY-DEEP-LEARNING-for-algorithmic-trading-using-Python-main" 



def feature_engineering(df):
    """ DON'T PUT THE SHIFT HERE"""
    
    # We copy the dataframe to avoid interferences in the data
    df_copy = df.copy()
    df_copy[["returns"]] = df_copy[["close"]].pct_change(1)

    # Create the SMAs
    df_copy["SMA 15"] = df_copy[["close"]].rolling(15).mean()
    df_copy["SMA 60"] = df_copy[["close"]].rolling(60).mean()
    
    # Create the volatilities
    df_copy["MSD 10"] = df_copy[["returns"]].rolling(10).std()
    df_copy["MSD 30"] = df_copy[["returns"]].rolling(30).std()
    
    # Create the Rsi
    RSI = ta.momentum.RSIIndicator(df_copy["close"], window= 14, fillna = False)
    df_copy["rsi"] = RSI.rsi()
    ta.add_all_ta_features(df_copy, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
    
    # STANDARDIZATION

    sc = StandardScaler()
    df_copy = df_copy[['SMA 15', 'SMA 60',
           'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi', 'volume_em',
           'volume_sma_em', 'volume_vpt', 'volume_vwap', 'volume_mfi',
           'volume_nvi', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
           'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
           'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
           'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
           'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
           'volatility_dcw', 'volatility_dcp', 'volatility_atr', 'volatility_ui',
           'trend_macd', 'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
           'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow',
           'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
           'trend_trix', 'trend_mass_index', 'trend_dpo', 'trend_kst',
           'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
           'trend_ichimoku_base', 'trend_ichimoku_a', 'trend_ichimoku_b',
           'trend_stc', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_cci',
           'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b', 'trend_aroon_up',
           'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
           'trend_psar_down', 'trend_psar_up_indicator',
           'trend_psar_down_indicator', 'momentum_rsi', 'momentum_stoch_rsi',
           'momentum_stoch_rsi_k', 'momentum_stoch_rsi_d', 'momentum_tsi',
           'momentum_uo', 'momentum_stoch', 'momentum_stoch_signal', 'momentum_wr',
           'momentum_ao', 'momentum_roc', 'momentum_ppo', 'momentum_ppo_signal',
           'momentum_ppo_hist', 'momentum_pvo', 'momentum_pvo_signal',
           'momentum_pvo_hist', 'momentum_kama', 'others_dr', 'others_dlr',
           'others_cr', 'MSD 10', 'MSD 30', 'rsi']].dropna()
    df_copy_sc = sc.fit_transform(df_copy)

    return df_copy_sc

def X_3d_RNN(X_s, y_s, lag):
    # Simple verification
    if len(X_s) != len(y_s):
        print("Warnings")
    
    # Create the X_train
    X_train = []
    for variable in range(X_s.shape[1]):
        X = []
        for i in range(lag, X_s.shape[0]):
            X.append(X_s[i-lag:i, variable])
        X_train.append(X)
    X_train = np.array(X_train)
    X_train = np.swapaxes(X_train, 0, 1)
    X_train = np.swapaxes(X_train, 1, 2)
    
    # Create the y_train
    y_train = []
    for i in range(lag, y_s.shape[0]):
        y_train.append(y_s[i, :].reshape(-1, 1).transpose())
    y_train = np.concatenate(y_train, axis=0)
    
    return X_train, y_train

def RNN_weights(pct_dropout = 0.5):
    # INITIALIZATION OF THE MODEL
    classifier = Sequential()

    
    # ADD LSTM LAYER
    classifier.add(LSTM(units = 50, return_sequences = True,
                      input_shape = (15,91,)))    
    classifier.add(Dropout(pct_dropout)) 
    


    # LOOP WHICH ADD LSTM LAYER
    for _ in range(3):
        classifier.add(Conv1D(64,3, activation='relu'))
        classifier.add(LSTM(units = 50, return_sequences = True))
        classifier.add(Dropout(pct_dropout))

    # LAST LSTM LAYER BUT WITH return_sequences = False 
    classifier.add(LSTM(units = 50, return_sequences = False))
    classifier.add(Dropout(pct_dropout))


    # OUTPUT DENSE LAYER
    classifier.add(Dense(1, activation="sigmoid"))

    # COMPILE THE MODEL
    classifier.compile(loss="binary_crossentropy", optimizer="adam")

    return classifier

def RNN_cl_sig(symbol):
    """ Function for predict the value of tommorow using DNN model"""
    
    # Create the weights if there is not in the folder
    rnn_1 = RNN_weights()
    rnn_2 = RNN_weights()
    rnn_3 = RNN_weights()

    # Import trained weights
    rnn_1.load_weights(f"Weights_RCNN_BTC_10M/RCNN n°27")
    rnn_2.load_weights(f"Weights_RCNN_BTC_10M/RCNN n°50")
    rnn_3.load_weights(f"Weights_RCNN_BTC_10M/RCNN n°130")
    
    # Take the lastest percentage of change 
    df = MT5.get_data(symbol, 700)[["open", "high", "low", "close", "tick_volume"]]
    df.columns = ["open", "high", "low", "close", "volume"]  
    
    # Features engeeniring
    data = feature_engineering(df)
    X_data, _ = X_3d_RNN(data, np.zeros([700,1]),15)
    X = X_data[-1:,:,:]
    print(np.shape(X))
    # Bagging
    pr1 = np.where(rnn_1.predict(X)==0, -1,1)
    pr2 = np.where(rnn_2.predict(X)==0, -1,1)
    pr3 = np.where(rnn_3.predict(X)==0, -1,1)
    
    # Find the signal
    buy = (pr1 + pr2 + pr3)[0][0] >=1
    sell = not buy
        
     
    return buy, sell

# True = Live Trading and False = Screener
live = True


if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")  
    print("------------------------------------------------------------------")
    


info_order = {
    "WIN": ["BTCUSD", "BTC-RCNN-M10"]
}


start = datetime.now().strftime("%H:%M:%S")
while True:
    if datetime.now().weekday() not in ():
        is_time = datetime.now().strftime("%H:%M:%S") >= "00:00:00" and datetime.now().strftime("%H:%M:%S") <= "23:59:00"
     
        
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = RNN_cl_sig(symbol)

              # Run the algorithm
            if live:
                
                MT5.run(symbol, buy, sell,lot)
                time.sleep(600)
                

                

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
    time.sleep(1)

------------------------------------------------------------------
Date:  2023-06-20 19:58:54
Balance: 5434.11 USD, 	Equity: 5458.08 USD, 	Profit: 23.97 USD
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 19:59:02
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 20:09:13
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 2s 2s/step
------------------------------------------------------------------
Date:  2023-06-20 20:19:24
SYMBOL: BTCUSD
BUY:

OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 20:29:35
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 20:39:46
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------


Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 20:49:57
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 21:00:08
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------
Warnings
(1, 15, 91)
1/1 [==============================] - 1s 1s/step
------------------------------------------------------------------
Date:  2023-06-20 21:10:19
SYMBOL: BTCUSD
BUY: True 	  SHORT: False
OPEN LONG TRADE: ['1']
------------------------------------------------------------------


In [ ]:
import csv
res = ['0']
with open('C:\\Users\\Adminx\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common\\Files\\Sinal_BTC-RCNN-M10.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(res) 

In [ ]:
rnn_1.summary()

In [ ]:
is_time